# funcX Tutorial

funcX is a Function-as-a-Service (FaaS) platform for science that enables you to register functions in a cloud-hosted service and then reliably execute those functions on a remote funcX endpoint. This tutorial is configured to use a tutorial endpoint hosted by the funcX team. You can setup and use your own endpoint by following the [funcX documentation](https://funcx.readthedocs.io/en/latest/endpoints.html).

## funcX Python SDK

The funcX Python SDK provides programming abstractions for interacting with the funcX service. Before running this tutorial you should first install the funcX SDK as follows:

    $ pip install funcx

The funcX SDK exposes a `FuncXClient` and `FuncXExecutor` for all interactions with the funcX service. In order to use the funcX service, you must first authenticate using one of hundreds of supported identity provides (e.g., your institution, ORCID, Google).  As part of the authentication process you must grant permission for funcX to access your identity information (to retrieve your email address), Globus Groups management access (to share functions and endpoints), and Globus Search (to discover functions and endpoints).

In [1]:
from funcx import FuncXClient, FuncXExecutor

fx = FuncXExecutor(FuncXClient())
print("FuncXExecutor : ", fx)

FuncXExecutor :  <funcx.sdk.executor.FuncXExecutor object at 0x7f154dc67220>


# Basic usage

The following example demonstrates how you can execute a function with the `FuncXExecutor` interface.


### Submitting a function

At the most basic level, to invoke a function, you must provide:

 * the function 
 * the function arguments if any
 * the `endpoint_id` of the endpoint on which you wish to execute that function. 
    
> **Note**: Here we use the public funcX tutorial endpoint. You may change the `endpoint_id` to the UUID of any endpoint for which you have permission to execute functions.

In [2]:
# Define the function for remote execution
def hello_world():
    return "Hello World!"

tutorial_endpoint = '4b116d3c-1703-4f8f-9f6f-39921e5864df' # Public tutorial endpoint
future = fx.submit(hello_world, endpoint_id=tutorial_endpoint)

print("Submit returned: ", future)

Submit returned:  <FuncXFuture at 0x7f154dc67c40 state=pending>


### Getting results

Once you `submit` a function to the `FuncXExecutor`, the executor will return an instance of `FuncXFuture` in lieu of the result from the function.

**@kyle please help simplify the next para (copied over from parsl docs)**

When a normal Python function is invoked, the Python interpreter waits for the function to complete execution and returns the results. In case of long running functions, it may not be desirable to wait for completion. Instead, it is preferable that functions are executed asynchronously. funcX provides such asynchronous behavior by returning a future in lieu of results. A future is essentially an object that allows funcX to track the status of an asynchronous task so that it may, in the future, be interrogated to find the status, results, exceptions, etc.

`FuncXFuture`s returned from the `FuncXExecutor` can be used in the following ways:
* `future.done()` is non-blocking and returns a boolean that indicates whether the task is done
* `future.result()` is a blocking call, that returns the result from the remote execution or raises an exception in case task execution failed. 

In [3]:
# Returns a boolean that indicates task completion
future.done()

False

In [ ]:
# Waits for the function to complete and returns the task result or exception on failure
future.result()

### Catching exceptions

When a function fails and you try to get its result, the `future` will raise an exception. In the following example, the 'deterministic failure' exception is raised when `future.result()` is called.

In [ ]:
def division_by_zero():
    return 42 / 0 # This will raise a 

future = fx.submit(division_by_zero, endpoint_id=tutorial_endpoint)

# This will raise an exception
try:
    future.result()
except Exception as exc:
    print("funcX returned an exception: ", exc)

## Functions with arguments

funcX supports registration and invocation of functions with arbitrary arguments and returned parameters. funcX will serialize any `*args` and `**kwargs` when invoking a function and it will serialize any return parameters or exceptions.  Note: funcX uses standard Python serilaization libraries (e.g., Pickle, Dill). It also limits the size of input arguments and returned parameters to 5 MB. 

The following example shows a function that computes the sum of a list of input arguments. First, we register the function as above:

In [ ]:
def funcx_sum(a, b):
    return a + b

future = fx.submit(funcx_sum, 40, 2, endpoint_id=tutorial_endpoint)
print(f"funcX returns 40 + 2 = {future.result()}")


## Functions with dependencies

funcX requires that functions explictly state all dependencies within the function body. It also assumes that the dependent libraries are available on the endpoint in which the function will execute.  For example, in the following function, we explictly import the time module. 

In [ ]:
def funcx_date():
    from datetime import date
    return date.today()

future = fx.submit(funcx_date, endpoint_id=tutorial_endpoint)

print("Date fetched from endpoint: ", future.result())

## Calling external applications

Depending on the configuration of the funcX endpoint, you can often invoke external applications that are available in the endpoint environment. 


In [ ]:
def funcx_echo(name):
    import os
    return os.popen("echo Hello {} from $HOSTNAME".format(name)).read()

future = fx.submit(funcx_echo, "World", endpoint_id=tutorial_endpoint)

print("Echo output: ", future.result())

## Running functions many times

After registering a function you can invoke it repeatedly. The following example shows how the Monte Carlo method can be used to estimate pi. 

Specifically, if a circle with radius $r$ is inscribed inside a square with side length $2r$, the area of the circle is $\pi r^2$ and the area of the square is $(2r)^2$. Thus, if $N$ uniformly-distributed points are dropped at random locations within the square, approximately $N\pi/4$ will be inside the circle.


In [ ]:
import time

# function that estimates pi by placing points in a box
def pi(num_points):
    from random import random
    inside = 0   
    
    return 3 # BADD
    for i in range(num_points):
        x, y = random(), random()  # Drop a point randomly within the box.
        if x**2 + y**2 < 1:        # Count points within the circle.
            inside += 1  
    return (inside*4 / num_points)


# execute the function 3 times 
estimates = []
for i in range(3):
    estimates.append(fx.submit(pi, 
                               10**5, 
                               endpoint_id=tutorial_endpoint))

print(estimates)
# get the results and calculate the total
# total = [future.result() for future in estimates]
for future in estimates:
    print(future.result())

print("Total : ", total)

# print the results
print("Estimates: {}".format(results))
print("Average: {:.5f}".format(total/len(results)))

# Endpoint operations

You can retrieve information about endpoints including status and information about how the endpoint is configured. 

In [ ]:
from funcx import FuncXClient
fxc = FuncXClient()

fxc.get_endpoint_status(tutorial_endpoint)